**Jupyter notebook**: Place your cursor in each In field and press Shift-Enter to execute it.


In [1]:
system"cd ",getenv[`HOME],"/course-introductory-workshop"
.trn.nbdir:system"cd"
\l scripts/loaddata.q

"Initializing variables"
"Loaded Weather CSV"
"Loaded Taxi Trips partitioned DB"
"Defining exercise results"
"Ready"


**Learning objectives** 

To understand:
* How to construct a qSQL query
* How to assign a variable
* Useful inbuilt aggregations 
* Grouping with by
* Using `fby` to avoid nested queries
* Updating existing data 
* Temporal arithmetic 

# Data overview

The taxi database details cab fares in New York City. The data was provided from the
[NYC Taxi & Limousine Commission](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page).

The database we have loaded contains trip data starting at January 1st, 2009 until March 31st, 2009. The
database contains 6.9 million records and occupies 6Gb on disk with no compression. 

The taxi database is stored in a table called `trips`. To supplement the data, there is an additional New York City weather dataset called `weather`. This data was collected by the  [National Oceanic and Atmospheric Administration](https://www.noaa.gov) and distributed by the 
[National Climate and Data Center](https://www.ncdc.noaa.gov). The weather dataset contains daily
summaries for January 2009 from 187 distinct weather stations surrounding New York City.

# Data exploration

Tables form the core of Kx technology. Here we will go through how to access tables and examine their contents.

See what tables are currently in the database:

In [2]:
tables[]

`smalltrips`trips`weather


Inspect a table by typing its name.

In [3]:
smalltrips

date       month   vendor pickup_time                   dropoff_time         ..
-----------------------------------------------------------------------------..
2009.01.01 2009.01 CMT    2009.01.01D00:00:00.000000000 2009.01.01D00:04:12.0..
2009.01.01 2009.01 CMT    2009.01.01D00:00:00.000000000 2009.01.01D00:05:03.0..
2009.01.01 2009.01 CMT    2009.01.01D00:00:02.000000000 2009.01.01D00:05:40.0..
2009.01.01 2009.01 CMT    2009.01.01D00:00:04.000000000 2009.01.01D00:03:08.0..
2009.01.01 2009.01 CMT    2009.01.01D00:00:07.000000000 2009.01.01D00:19:01.0..
2009.01.01 2009.01 CMT    2009.01.01D00:00:09.000000000 2009.01.01D00:05:43.0..
2009.01.01 2009.01 CMT    2009.01.01D00:00:10.000000000 2009.01.01D00:05:50.0..
2009.01.01 2009.01 CMT    2009.01.01D00:00:13.000000000 2009.01.01D00:15:27.0..
2009.01.01 2009.01 DDS    2009.01.01D00:00:15.000000000 2009.01.01D00:01:29.0..
2009.01.01 2009.01 CMT    2009.01.01D00:00:16.000000000 2009.01.01D00:12:02.0..
2009.01.01 2009.01 CMT    2009.01.01D00:

It is very easy to interrogate the kdb+/q process to get information from the process about the size and shape of the tables.

We can see how many records are in the table, by using [`count`](https://code.kx.com/q/ref/count).

We can also inspect the schema of `trips` with [`meta`](https://code.kx.com/q/ref/meta), obtaining a table of available columns and their types and other information. The following columns are produced:

- `c`: column name
- `t`: column [type](https://code.kx.com/q/ref/#datatypes)
- `f`: [foreign keys](https://code.kx.com/q/wp/foreign-keys/)
- `a`: [attributes](https://code.kx.com/q/basics/syntax/#attributes): modifiers applied for performance characteristics

In [4]:
count weather
cols weather
meta weather

31


`date`maxtemp`mintemp`avgtemp`departuretemp`hdd`cdd`precip`newsnow`snowdepth


c            | t f a
-------------| -----
date         | d    
maxtemp      | f    
mintemp      | f    
avgtemp      | f    
departuretemp| f    
hdd          | f    
cdd          | f    
precip       | f    
newsnow      | f    
snowdepth    | f    


# qSQL

If you have used SQL, you will find the syntax of qSQL queries very similar.

 <img src="images/qbies.png" width="50px" align="left"/><p style='color:#273a6e'><i> Here qSQL is like SQL but does not require a <code>*</code> to select all columns. </i></p> 

In [19]:
select from weather

date       maxtemp mintemp avgtemp departuretemp hdd cdd precip newsnow snowd..
-----------------------------------------------------------------------------..
2009.01.01 26      15      20.5    -12.9         44  0   0      0       0    ..
2009.01.02 34      23      28.5    -4.8          36  0                  0    ..
2009.01.03 38      29      33.5    0.4           31  0                  0    ..
2009.01.04 42      25      33.5    0.5           31  0   0      0       0    ..
2009.01.05 43      38      40.5    7.6           24  0          0       0    ..
2009.01.06 38      31      34.5    1.7           30  0   0.08           0    ..
2009.01.07 38      31      34.5    1.8           30  0   1.19   0       0    ..
2009.01.08 38      29      33.5    0.9           31  0   0      0       0    ..
2009.01.09 32      26      29      -3.5          36  0   0      0       0    ..
2009.01.10 30      23      26.5    -5.9          38  0   0.14   1       0    ..
2009.01.11 31      24      27.5    -4.9 

Get the `vendor`, `pickup_time` and `fare` columns from the entire table.

In [20]:
select vendor, pickup_time, fare from smalltrips

vendor pickup_time                   fare
-----------------------------------------
CMT    2009.01.01D00:00:00.000000000 5.8 
CMT    2009.01.01D00:00:00.000000000 5.4 
CMT    2009.01.01D00:00:02.000000000 5.8 
CMT    2009.01.01D00:00:04.000000000 4.6 
CMT    2009.01.01D00:00:07.000000000 27.8
CMT    2009.01.01D00:00:09.000000000 5.8 
CMT    2009.01.01D00:00:10.000000000 5.8 
CMT    2009.01.01D00:00:13.000000000 10.6
DDS    2009.01.01D00:00:15.000000000 2.9 
CMT    2009.01.01D00:00:16.000000000 11.8
CMT    2009.01.01D00:00:17.000000000 20.2
CMT    2009.01.01D00:00:21.000000000 23.4
CMT    2009.01.01D00:00:23.000000000 12.2
CMT    2009.01.01D00:00:25.000000000 17.8
CMT    2009.01.01D00:00:27.000000000 30.2
CMT    2009.01.01D00:00:27.000000000 7.4 
CMT    2009.01.01D00:00:27.000000000 5.8 
CMT    2009.01.01D00:00:28.000000000 9   
CMT    2009.01.01D00:00:29.000000000 14.6
DDS    2009.01.01D00:00:29.000000000 17.3
..


The next query extracts a few columns, but also filters on two criteria. Just as in SQL, table results can be filtered by expressions following a `where`. Multiple filter criteria, separated by `,`, are evaluated starting from the left.

Get taxi trips from the first day of data where the tip is greater than 20.

 <img src="images/qbies.png" width="50px" align="left"/><p style='color:#273a6e'><i> The first day of data is found by applying aggregation <code>min</code> to the <code>date</code> column, and selecting dates equal to it. <br><br> Q has many aggregations. We shall encounter more later. </i></p> 

In [21]:
select date, month, vendor, passengers, fare, tip from trips 
    where date = min date, tip > 50   

date       month   vendor passengers fare tip  
-----------------------------------------------
2009.01.01 2009.01 CMT    1          47   95.45
2009.01.01 2009.01 CMT    1          45   60   
2009.01.01 2009.01 CMT    1          45   58.88
2009.01.01 2009.01 CMT    1          45   57   
2009.01.01 2009.01 VTS    2          5.3  98.23
2009.01.01 2009.01 DDS    1          2.5  58   
2009.01.01 2009.01 VTS    1          2.5  67   
2009.01.01 2009.01 DDS    1          45   90   


**Structure of kdb+ database**

<img src="images/DbStructure.png" width="600" height="400">

Which query runs faster?

\ts select date, month, vendor, passengers, fare, tip from trips where date = min date, tip > 20
\ts select date, month, vendor, passengers, fare, tip from trips where tip > 20, date = min date

(1) runs faster. The `trips` table is a partitioned on the `date` column, which is the first filter in the [Where phrase](https://code.kx.com/q/basics/qsql/#where-phrase). Filtering on the partition column value first minimizes the number of directories to read.

We can test this theory with the [`\ts`](https://code.kx.com/q/basics/syscmds/#ts-time-and-space) system command.

In [22]:
\ts select date, month, vendor, passengers, fare, tip from trips where date = min date, tip > 20
\ts select date, month, vendor, passengers, fare, tip from trips where tip > 20, date = min date

42 794016


4053 1725040


Include calculated columns in the result by assignment in the form `column_name : value`.

In [20]:
select date, month, vendor, passengers, fare, tip from trips where date = min date, tip > 20
select date, month, vendor, passengers, fare, tip from trips where tip > 20, date = min date

date       month   vendor passengers fare tip  
-----------------------------------------------
2009.01.01 2009.01 DDS    1          33.3 33.8 
2009.01.01 2009.01 CMT    2          77.4 30   
2009.01.01 2009.01 CMT    1          47   95.45
2009.01.01 2009.01 CMT    1          112  23.6 
2009.01.01 2009.01 CMT    1          41.4 40   
2009.01.01 2009.01 CMT    2          26.2 36.23
2009.01.01 2009.01 CMT    3          27.4 20.8 
2009.01.01 2009.01 CMT    3          135  25   
2009.01.01 2009.01 DDS    4          127  23   
2009.01.01 2009.01 CMT    1          11   24.24
2009.01.01 2009.01 CMT    4          27.8 24.24
2009.01.01 2009.01 CMT    1          120  24   
2009.01.01 2009.01 CMT    4          10.6 28.84
2009.01.01 2009.01 CMT    1          9    22   
2009.01.01 2009.01 CMT    2          15.4 30   
2009.01.01 2009.01 CMT    1          24.2 30   
2009.01.01 2009.01 CMT    1          13   20.2 
2009.01.01 2009.01 CMT    1          45   60   
2009.01.01 2009.01 DDS    1          136

date       month   vendor passengers fare tip  
-----------------------------------------------
2009.01.01 2009.01 DDS    1          33.3 33.8 
2009.01.01 2009.01 CMT    2          77.4 30   
2009.01.01 2009.01 CMT    1          47   95.45
2009.01.01 2009.01 CMT    1          112  23.6 
2009.01.01 2009.01 CMT    1          41.4 40   
2009.01.01 2009.01 CMT    2          26.2 36.23
2009.01.01 2009.01 CMT    3          27.4 20.8 
2009.01.01 2009.01 CMT    3          135  25   
2009.01.01 2009.01 DDS    4          127  23   
2009.01.01 2009.01 CMT    1          11   24.24
2009.01.01 2009.01 CMT    4          27.8 24.24
2009.01.01 2009.01 CMT    1          120  24   
2009.01.01 2009.01 CMT    4          10.6 28.84
2009.01.01 2009.01 CMT    1          9    22   
2009.01.01 2009.01 CMT    2          15.4 30   
2009.01.01 2009.01 CMT    1          24.2 30   
2009.01.01 2009.01 CMT    1          13   20.2 
2009.01.01 2009.01 CMT    1          45   60   
2009.01.01 2009.01 DDS    1          136

A virtual column `i` maps to a record index in the table. A simple aggregation can be obtained by taking the count of this virtual column.

In [24]:
select count i from trips where date = min date, passengers = 4

x    
-----
17029


Further reading:
-   [qSQL syntax](https://code.kx.com/q/basics/qsql/)
-   [*Q for Mortals* §9: Queries](https://code.kx.com/q4m3/9_Queries_q-sql/)

##### Exercise 1

- On the earliest date, how many trips had fewer than two passengers?

In [5]:
select count i from trips where date = min date, passengers < 2
//alternative way- select count i from trips where date = first date, passengers < 2

x     
------
183492


- What about the last date?

In [21]:
select count i from trips where date = max date, passengers < 2
//alternative way- select count i from trips where date = last date, passengers < 2

x     
------
332941


In [6]:
select payment_type, fare from trips where date = min date


payment_type fare
-----------------
CASH         5.8 
CASH         5.4 
CASH         5.8 
CASH         4.6 
CASH         27.8
CASH         5.8 
CASH         5.8 
CASH         10.6
CASH         2.9 
CASH         11.8
CASH         20.2
CASH         23.4
CASH         12.2
CASH         17.8
CASH         30.2
CASH         7.4 
CASH         5.8 
CASH         9   
No Charge    14.6
CASH         17.3
..


In [7]:
ex1_b[] //check correct output

x     
------
332941


##### Exercise 2
- Select `payment_type` and `fare` from the trips data for the first date.


In [8]:
select payment_type, fare from trips where date = min date

payment_type fare
-----------------
CASH         5.8 
CASH         5.4 
CASH         5.8 
CASH         4.6 
CASH         27.8
CASH         5.8 
CASH         5.8 
CASH         10.6
CASH         2.9 
CASH         11.8
CASH         20.2
CASH         23.4
CASH         12.2
CASH         17.8
CASH         30.2
CASH         7.4 
CASH         5.8 
CASH         9   
No Charge    14.6
CASH         17.3
..


In [ ]:
// Enter your qSQL code here 

In [10]:
ex2[] //check correct output

payment_type fare
-----------------
CASH         5.8 
CASH         5.4 
CASH         5.8 
CASH         4.6 
CASH         27.8
CASH         5.8 
CASH         5.8 
CASH         10.6
CASH         2.9 
CASH         11.8
CASH         20.2
CASH         23.4
CASH         12.2
CASH         17.8
CASH         30.2
CASH         7.4 
CASH         5.8 
CASH         9   
No Charge    14.6
CASH         17.3
..


## Assignment

When exploring data, storing intermediate results is useful for avoiding repeated work.

 <img src="images/qbies.png" width="50px" align="left"/><p style='color:#273a6e'><i> In q, assignment is denoted by <code>:</code>.<br><br>The equal sign <code>=</code> retains its traditional meaning of testing for equality. </i></p>

Create a new variable as follows:

```
var_name: expression
```

In order to take a closer look at only the data corresponding to January 2009, we can select the data from trips and store the resulting table in a variable called `jan09`.

In [50]:
// Use keyword 'within' to filter the date
jan09:select from trips where date within 2009.01.10 2009.01.31

// Check how many records are in the filtered table
count jan09

10420159


 <img src="images/qbies.png" width="50px" align="left"/><p style='color:#273a6e'><i> By making this assignment, the new variable <code>jan09</code> is stored in memory. <br><br> This is true whether the source data was also in memory or stored on disk. </i></p>

##### Exercise 3

+ Assign the result of Exercise 2 to a variable called `res2`

In [51]:
res2:select payment_type, fare from trips where date = min date;

In [ ]:
// Enter your qSQL code here 

In [16]:
ex3[] //check correct output

payment_type fare
-----------------
CASH         5.8 
CASH         5.4 
CASH         5.8 
CASH         4.6 
CASH         27.8
CASH         5.8 
CASH         5.8 
CASH         10.6
CASH         2.9 
CASH         11.8
CASH         20.2
CASH         23.4
CASH         12.2
CASH         17.8
CASH         30.2
CASH         7.4 
CASH         5.8 
CASH         9   
No Charge    14.6
CASH         17.3
..


## Aggregations

With the filtered data in `jan09`, aggregations and other analytics can be run without performing the filter query again. Below is a simple aggregation returning the sum of both the `fare` and `tip` columns. 

In [17]:
select sum fare, sum tip from jan09

fare         tip    
--------------------
9.840941e+07 5005536


[`sum`](https://code.kx.com/q/ref/sum/) is one of many built-in aggregations. Other built-in aggregations include, but are not limited to

- [`avg`](https://code.kx.com/q/ref/avg/#avg) - average (mean)
- [`med`](https://code.kx.com/q/ref/med/) - median
- [`min`](https://code.kx.com/q/ref/min/) - minimum value
- [`max`](https://code.kx.com/q/ref/max/) - maximum value
- [`count`](https://code.kx.com/q/ref/count/) - number of values

Reference: [Mathematics and statistics](https://code.kx.com/q/basics/math/)

##### Exercise 4
- Calculate the minimum and maximum tip from the `jan09` table.


In [18]:
// maximum tip and minimum tip
select maxTip:max tip, minTip:min tip from jan09

maxTip minTip
-------------
100    0     


In [ ]:
// Enter your qSQL code here 

In [19]:
ex4[] //check correct output

maxTip minTip
-------------
100    0     


## Grouping with `by` 

Unlike SQL, qSQL lets you group and aggregate separately.
The easiest way to group similar values together is to use the `by` clause.

When used without an aggregation, `by` returns a list of values from the selected column. List types are completely valid types in q, so this fine.

In [20]:
select fare by vendor from jan09

vendor| fare                                                                 ..
------| ---------------------------------------------------------------------..
CMT   | 6.2 12.6 7 6.6 11 12.2 6.6 14.2 7 9 13 7.4 10.6 10.2 15.8 40.2 9.4 5...
DDS   | 10.9 8.9 9.7 10.9 10.1 6.1 10.9 7.7 8.5 12.1 18.9 10.5 18.5 11.3 9.7 ..
VTS   | 11.3 15.7 18.1 15.7 6.5 4.5 4.5 15.3 4.9 4.9 4.5 6.9 18.5 20.1 14.1 6..


To extract some useful insight from the data, grouping is best used with aggregation functions as shown below:

In [22]:
select sum fare, sum tip by vendor from jan09

vendor| fare         tip     
------| ---------------------
CMT   | 4.430574e+07 2059982 
DDS   | 6120686      273323.2
VTS   | 4.798299e+07 2672231 


In [23]:
// Get the number of records per day in the filtered table
select count i by date from jan09

date      | x     
----------| ------
2009.01.10| 483350
2009.01.11| 405075
2009.01.12| 414642
2009.01.13| 442543
2009.01.14| 489177
2009.01.15| 486450
2009.01.16| 535200
2009.01.17| 511023
2009.01.18| 419962
2009.01.19| 352534
2009.01.20| 433639
2009.01.21| 478268
2009.01.22| 498241
2009.01.23| 519394
2009.01.24| 547591
2009.01.25| 460723
2009.01.26| 434081
2009.01.27| 471788
2009.01.28| 455187
2009.01.29| 503352
..


##### Exercise 5
- What is the biggest tip for each vendor?

In [24]:
select max tip by vendor from jan09

vendor| tip  
------| -----
CMT   | 93.22
DDS   | 100  
VTS   | 100  


In [ ]:
// Enter your qSQL code here 

In [25]:
ex5[] //check correct output

vendor| tip  
------| -----
CMT   | 93.22
DDS   | 100  
VTS   | 100  


##### Exercise 6 
- What is the highest tip and average tip per payment_type?

In [27]:
select maxTip:max tip, avgTip:avg tip by payment_type from jan09

payment_type| maxTip avgTip     
------------| ------------------
CASH        | 82     0.00077795 
CREDIT      | 100    2.145682   
Dispute     | 11.25  0.01481096 
No Charge   | 13.35  0.006573717


In [ ]:
// Enter your qSQL code here 

In [28]:
ex6[] //check correct output

payment_type| maxTip avgTip     
------------| ------------------
CASH        | 82     0.00077795 
CREDIT      | 100    2.145682   
Dispute     | 11.25  0.01481096 
No Charge   | 13.35  0.006573717


## Using `fby` to avoid nested queries

Nested queries are commonly required in SQL where filter criteria require aggregations in the context of some other column. For example, getting all records where the ride duration is less than the average for that taxi’s vendor.

In SQL  we would first get the average duration for each vendor; then perform a query for each duration and vendor:

In [29]:
// Get the average duration per vendor and save resulting table in a variable
resBy: select avgDuration:avg duration by vendor from jan09

In [30]:
// Using 'lj' to join the average duration column to our table
// Don't worry about this now, joins are discussed in a later section
select from jan09 lj resBy where duration < avgDuration

date       month   vendor pickup_time                   dropoff_time         ..
-----------------------------------------------------------------------------..
2009.01.10 2009.01 VTS    2009.01.10D00:00:00.000000000 2009.01.10D00:08:00.0..
2009.01.10 2009.01 VTS    2009.01.10D00:00:00.000000000 2009.01.10D00:04:00.0..
2009.01.10 2009.01 VTS    2009.01.10D00:00:00.000000000 2009.01.10D00:02:00.0..
2009.01.10 2009.01 VTS    2009.01.10D00:00:00.000000000 2009.01.10D00:05:00.0..
2009.01.10 2009.01 VTS    2009.01.10D00:00:00.000000000 2009.01.10D00:08:00.0..
2009.01.10 2009.01 VTS    2009.01.10D00:00:00.000000000 2009.01.10D00:07:00.0..
2009.01.10 2009.01 VTS    2009.01.10D00:00:00.000000000 2009.01.10D00:07:00.0..
2009.01.10 2009.01 VTS    2009.01.10D00:00:00.000000000 2009.01.10D00:03:00.0..
2009.01.10 2009.01 VTS    2009.01.10D00:00:00.000000000 2009.01.10D00:09:00.0..
2009.01.10 2009.01 VTS    2009.01.10D00:00:00.000000000 2009.01.10D00:03:00.0..
2009.01.10 2009.01 VTS    2009.01.10D00:

In q, this can be simply expressed using [`fby`](https://code.kx.com/q/ref/fby/).

The syntax of `fby` is `(aggregation;data) fby group`, as query below. Compare the above statement to how it was done without `fby`.

In [31]:
select max fare from jan09 where duration < (avg;duration) fby vendor

fare
----
200 


You can see how much simpler an `fby` clause is compared to the above statements.

##### Exercise 7 

- Which payment type produces the highest average tip when only trips with a fare larger than the average for each vendor is considered?

In [32]:
res7a:select avg tip by payment_type from jan09 where fare > (avg;fare) fby vendor;
show res7a;
select payment_type from res7a where tip = max tip 

payment_type| tip        
------------| -----------
CASH        | 0.001723333
CREDIT      | 3.004373   
Dispute     | 0.02912136 
No Charge   | 0.0112184  


payment_type
------------
CREDIT      


In [ ]:
// Enter your qSQL code here 

In [33]:
ex7_a[] //check correct output

payment_type| tip        
------------| -----------
CASH        | 0.001723333
CREDIT      | 3.004373   
Dispute     | 0.02912136 
No Charge   | 0.0112184  


payment_type
------------
CREDIT      


- Which vendor has the largest number of trips when only considering trips shorter than the average duration for each vendor?

In [34]:
res7b:select count i by vendor from jan09 where duration < (avg;duration) fby vendor;
show res7b;
select vendor from res7b where x = max x

vendor| x      
------| -------
CMT   | 2883225
DDS   | 402781 
VTS   | 3592460


vendor
------
VTS   


In [ ]:
// Enter your qSQL code here 

In [35]:
ex7_b[] //check correct output

vendor| x      
------| -------
CMT   | 2883225
DDS   | 402781 
VTS   | 3592460


vendor
------
VTS   


## Updating existing data

When exploring, you often need to fix bad data, whether from incorrect entry, incompatible units of measure, or other reasons. 

If we inspect the data and look at the maximum passengers for each vendor, the maximum number for 2 vendors is 6.

In [36]:
select max passengers by vendor from jan09

vendor| passengers
------| ----------
CMT   | 5         
DDS   | 6         
VTS   | 113       


If the above data is wrong, it can be fixed using `update` to change the data. Below, the passengers column is set to the value `5` in any entry where the passenger count is greater than 5. This is then assigned back to `jan09`.

In [37]:
// Find all records where the number of passengers is greater than 5
select from jan09 where passengers > 5

date       month   vendor pickup_time                   dropoff_time         ..
-----------------------------------------------------------------------------..
2009.01.10 2009.01 VTS    2009.01.10D00:01:00.000000000 2009.01.10D00:09:00.0..
2009.01.10 2009.01 VTS    2009.01.10D00:02:00.000000000 2009.01.10D00:05:00.0..
2009.01.10 2009.01 VTS    2009.01.10D00:02:00.000000000 2009.01.10D00:24:00.0..
2009.01.10 2009.01 VTS    2009.01.10D00:03:00.000000000 2009.01.10D00:12:00.0..
2009.01.10 2009.01 VTS    2009.01.10D00:03:00.000000000 2009.01.10D00:24:00.0..
2009.01.10 2009.01 VTS    2009.01.10D00:04:00.000000000 2009.01.10D00:28:00.0..
2009.01.10 2009.01 VTS    2009.01.10D00:04:00.000000000 2009.01.10D00:07:00.0..
2009.01.10 2009.01 VTS    2009.01.10D00:05:00.000000000 2009.01.10D00:28:00.0..
2009.01.10 2009.01 VTS    2009.01.10D00:05:00.000000000 2009.01.10D00:12:00.0..
2009.01.10 2009.01 VTS    2009.01.10D00:05:00.000000000 2009.01.10D00:30:00.0..
2009.01.10 2009.01 VTS    2009.01.10D00:

In [38]:
// Update maximum number of passengers to be 5 
// Save changes by reassigning back to jan09
jan09: update passengers: 5 from jan09 where passengers > 5

In [27]:
jan09

[0;31mjan09[0m: [0;31mjan09[0m

Observe below that the maximum passengers reported has now been updated.

In [39]:
select max passengers by vendor from jan09

vendor| passengers
------| ----------
CMT   | 5         
DDS   | 5         
VTS   | 5         


Now that we have updated the table, we can add a column with the weighted average fare per passenger:

In [40]:
jan09:update wAvgfare:passengers wavg fare from jan09
meta jan09 //new column has been added to the end of the table

c           | t f a
------------| -----
date        | d    
month       | m    
vendor      | s    
pickup_time | p    
dropoff_time| p    
duration    | n    
passengers  | i    
distance    | f    
start_long  | f    
start_lat   | f    
end_long    | f    
end_lat     | f    
payment_type| s    
fare        | f    
surcharge   | e    
tip         | f    
tolls       | f    
total       | f    
wAvgfare    | f    


We also see a few trips that didn't take place: no recorded duration. Let’s delete these rows:

In [41]:
count jan09  //number of records before deleting rows
jan09:delete from jan09 where duration=00:00:00.000
count jan09  //number of records after deleteing rows 

10420159


10357181


## Temporal Arithmetic

Q supports several temporal types and arithmetic between them. Writing queries involving time is simple.  

See [here](https://code.kx.com/q/ref/#datatypes) for a summary of [`datatypes`](https://code.kx.com/q/ref/#datatypes)

The `pickup_time` column in the data has a type of *timestamp*.
As an example, we could convert the `pickup_time` values to their `minute` values (including hours and minutes), and group the data based on this time frame.

In [42]:
select pickup_time, pickup_time.second, pickup_time.minute, pickup_time.hh from jan09

pickup_time                   second   minute hh
------------------------------------------------
2009.01.10D00:00:00.000000000 00:00:00 00:00  0 
2009.01.10D00:00:00.000000000 00:00:00 00:00  0 
2009.01.10D00:00:00.000000000 00:00:00 00:00  0 
2009.01.10D00:00:00.000000000 00:00:00 00:00  0 
2009.01.10D00:00:00.000000000 00:00:00 00:00  0 
2009.01.10D00:00:00.000000000 00:00:00 00:00  0 
2009.01.10D00:00:00.000000000 00:00:00 00:00  0 
2009.01.10D00:00:00.000000000 00:00:00 00:00  0 
2009.01.10D00:00:00.000000000 00:00:00 00:00  0 
2009.01.10D00:00:00.000000000 00:00:00 00:00  0 
2009.01.10D00:00:00.000000000 00:00:00 00:00  0 
2009.01.10D00:00:00.000000000 00:00:00 00:00  0 
2009.01.10D00:00:00.000000000 00:00:00 00:00  0 
2009.01.10D00:00:00.000000000 00:00:00 00:00  0 
2009.01.10D00:00:00.000000000 00:00:00 00:00  0 
2009.01.10D00:00:00.000000000 00:00:00 00:00  0 
2009.01.10D00:00:00.000000000 00:00:00 00:00  0 
2009.01.10D00:00:00.000000000 00:00:00 00:00  0 
2009.01.10D00:00:00.

 <img src="images/qbies.png" width="50px" align="left"/><p style='color:#273a6e'><i> The query below returns the total money made each minute by cabs in January. <br><br> Observe the following expression within the query: <code>sum fare + tip</code> </i></p>

This expression highlights an important feature of q syntax: **evaluation is from right-to-left**. The argument of `sum` is everything to its right, that is `fare` plus `tip. This simple rule holds everywhere; there are no priorities to remember.

In [43]:
select total:sum fare + tip by pickup_time.minute from jan09

minute| total   
------| --------
00:00 | 81036.05
00:01 | 80896.72
00:02 | 81054.25
00:03 | 79761.3 
00:04 | 79826.02
00:05 | 80357.91
00:06 | 79977.98
00:07 | 77943.35
00:08 | 79075.81
00:09 | 78359.53
00:10 | 80306.23
00:11 | 77727.6 
00:12 | 77619.98
00:13 | 76999.45
00:14 | 76558.47
00:15 | 76293.44
00:16 | 75827.81
00:17 | 78801.69
00:18 | 76313.8 
00:19 | 73535.35
..


In [44]:
select total:(sum fare) + tip by pickup_time.minute from jan09 

minute| total                                                                ..
------| ---------------------------------------------------------------------..
00:00 | 76879.25 76879.25 76879.25 76879.25 76881.25 76879.25 76882.25 76882...
00:01 | 76743.1 76748.45 76743.1 76743.1 76743.1 76743.1 76745.1 76745.1 7674..
00:02 | 77195.2 77195.2 77195.2 77196.2 77195.2 77195.2 77197.2 77195.2 77195..
00:03 | 75765.2 75763.2 75763.2 75763.2 75763.2 75763.2 75772.2 75763.2 75763..
00:04 | 75773.3 75773.3 75773.3 75773.3 75773.3 75773.3 75775.3 75773.3 75774..
00:05 | 76372.85 76371.85 76369.85 76369.85 76371.85 76369.85 76369.85 76372...
00:06 | 75990.96 75990.96 75990.96 75990.96 75990.96 75990.96 75990.96 75990...
00:07 | 74125.53 74124.23 74124.23 74124.23 74124.23 74124.23 74124.23 74124...
00:08 | 75002.25 75000.25 75000.25 75000.25 75001.25 75000.25 75002.25 75000...
00:09 | 74536.6 74540.32 74536.6 74536.6 74536.6 74540.6 74537.7 74536.6 7453..
00:10 | 76128.45 76128.45 76128.45 76128

Aggregate further on time by using `xbar` to bucket the minutes into hours. Group the minutes into 60-unit buckets to produce hours:

In [45]:
select count i by 60 xbar pickup_time.minute from jan09 where date = 2009.01.10

minute| x    
------| -----
00:00 | 28975
01:00 | 24007
02:00 | 20202
03:00 | 15472
04:00 | 9721 
05:00 | 4288 
06:00 | 4309 
07:00 | 6303 
08:00 | 9735 
09:00 | 14168
10:00 | 17338
11:00 | 19954
12:00 | 23246
13:00 | 25600
14:00 | 22418
15:00 | 23091
16:00 | 23762
17:00 | 22466
18:00 | 26649
19:00 | 30790
..


##### Exercise 8
- Show the largest tip for each 15-minute timespan during the month of January.

In [46]:
select max tip by 15 xbar pickup_time.minute from jan09

minute| tip  
------| -----
00:00 | 90   
00:15 | 68   
00:30 | 62   
00:45 | 100  
01:00 | 73.3 
01:15 | 57   
01:30 | 79   
01:45 | 58.25
02:00 | 93.22
02:15 | 55   
02:30 | 92   
02:45 | 100  
03:00 | 55   
03:15 | 67.54
03:30 | 85.25
03:45 | 90   
04:00 | 55.55
04:15 | 90.45
04:30 | 75   
04:45 | 65   
..


In [ ]:
// Enter your qSQL code here 

In [47]:
ex8_a[] //check correct output

minute| tip  
------| -----
00:00 | 90   
00:15 | 68   
00:30 | 62   
00:45 | 100  
01:00 | 73.3 
01:15 | 57   
01:30 | 79   
01:45 | 58.25
02:00 | 93.22
02:15 | 55   
02:30 | 92   
02:45 | 100  
03:00 | 55   
03:15 | 67.54
03:30 | 85.25
03:45 | 90   
04:00 | 55.55
04:15 | 90.45
04:30 | 75   
04:45 | 65   
..


 - Break this information down by vendor.

In [48]:
select max tip by 15 xbar pickup_time.minute,vendor from jan09

minute vendor| tip  
-------------| -----
00:00  CMT   | 35   
00:00  DDS   | 23.2 
00:00  VTS   | 90   
00:15  CMT   | 55   
00:15  DDS   | 55   
00:15  VTS   | 68   
00:30  CMT   | 39.98
00:30  DDS   | 24   
00:30  VTS   | 62   
00:45  CMT   | 55.33
00:45  DDS   | 24   
00:45  VTS   | 100  
01:00  CMT   | 55   
01:00  DDS   | 20   
01:00  VTS   | 73.3 
01:15  CMT   | 35.82
01:15  DDS   | 45.23
01:15  VTS   | 57   
01:30  CMT   | 70   
01:30  DDS   | 27   
..


In [ ]:
// Enter your qSQL code here 

In [49]:
ex8_b[] //check correct output

minute vendor| tip  
-------------| -----
00:00  CMT   | 35   
00:00  DDS   | 23.2 
00:00  VTS   | 90   
00:15  CMT   | 55   
00:15  DDS   | 55   
00:15  VTS   | 68   
00:30  CMT   | 39.98
00:30  DDS   | 24   
00:30  VTS   | 62   
00:45  CMT   | 55.33
00:45  DDS   | 24   
00:45  VTS   | 100  
01:00  CMT   | 55   
01:00  DDS   | 20   
01:00  VTS   | 73.3 
01:15  CMT   | 35.82
01:15  DDS   | 45.23
01:15  VTS   | 57   
01:30  CMT   | 70   
01:30  DDS   | 27   
..
